In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.5 MB/s 
     |████████████████████████████████| 7.6 MB 55.8 MB/s 
     |████████████████████████████████| 182 kB 78.5 MB/s 


In [2]:
!pip install textattack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 418 kB 4.5 MB/s 
     |████████████████████████████████| 125 kB 62.0 MB/s 
     |████████████████████████████████| 769 kB 75.5 MB/s 
     |████████████████████████████████| 365 kB 58.9 MB/s 
     |████████████████████████████████| 401 kB 67.4 MB/s 
     |████████████████████████████████| 60 kB 8.5 MB/s 
     |████████████████████████████████| 3.6 MB 58.5 MB/s 
     |████████████████████████████████| 41.4 MB 1.2 MB/s 
     |████████████████████████████████| 95 kB 4.1 MB/s 
     |████████████████████████████████| 212 kB 59.0 MB/s 
     |████████████████████████████████| 132 kB 61.4 MB/s 
     |████████████████████████████████| 127 kB 58.8 MB/s 
     |████████████████████████████████| 1.6 MB 56.7 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 788 kB 55.5 MB/s 
     |████████████████████████████████

In [3]:
import transformers
import textattack
import pandas as pd
from datasets import load_dataset
from transformers import AutoModel
from transformers import pipeline, AutoModelForTokenClassification
from transformers import AutoTokenizer

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# API transferability

In [4]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/xlnet-base-cased-rotten-tomatoes")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/xlnet-base-cased-rotten-tomatoes")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

Downloading:   0%|          | 0.00/801 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/469M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/202 [00:00<?, ?B/s]

In [5]:
dataset = textattack.datasets.HuggingFaceDataset("rotten_tomatoes", split="test")
attack = textattack.attack_recipes.BAEGarg2019.build(model_wrapper)
# Attack 20 samples with CSV logging and checkpoint saved every 5 interval
attack_args = textattack.AttackArgs(num_examples=20, log_to_csv="log.csv", checkpoint_interval=5, checkpoint_dir="checkpoints", disable_stdout=True)
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Dataset rotten_tomatoes downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

...

[Succeeded / Failed / Skipped / Total] 4 / 1 / 0 / 5:  25%|██▌       | 5/20 [00:42<02:06,  8.41s/it]textattack: Saving checkpoint under "checkpoints/1671388473030.ta.chkpt" at 2022-12-18 18:34:33 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 3 / 1 / 10:  50%|█████     | 10/20 [01:17<01:17,  7.73s/it]textattack: Saving checkpoint under "checkpoints/1671388508226.ta.chkpt" at 2022-12-18 18:35:08 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 3 / 4 / 15:  75%|███████▌  | 15/20 [01:29<00:29,  5.95s/it]textattack: Saving checkpoint under "checkpoints/1671388520233.ta.chkpt" at 2022-12-18 18:35:20 after 15 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 5 / 5 / 20: 100%|██████████| 20/20 [02:11<00:00,  6.56s/it]textattack: Saving checkpoint under "checkpoints/1671388562141.ta.chkpt" at 2022-12-18 18:36:02 after 20 attacks.
[Succeeded / Failed / Skipped / Total] 10 / 5 / 5 / 20: 100%|██████████| 20/20 [02:11<00:00,  6.56s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 10     |
| Number of failed attacks:     | 5      |
| Number of skipped attacks:    | 5      |
| Original accuracy:            | 75.0%  |
| Accuracy under attack:        | 25.0%  |
| Attack success rate:          | 66.67% |
| Average perturbed word %:     | 15.73% |
| Average num. words per input: | 17.5   |
| Avg num queries:              | 53.27  |
+-------------------------------+--------+


In [6]:
data = pd.read_csv('/content/log.csv')
data

,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
0,lovingly photographed in the manner of a golde...,lovingly photographed in the manner of a golde...,0.050576,0.672676,1,0,1,46,Successful
1,consistently [[clever]] and [[suspenseful]] .,consistently [[annoying]] and [[sarcastic]] .,0.000928,0.996569,1,0,1,35,Successful
2,"it's like a "" [[big]] chill "" reunion of the b...","it's like a "" [[bad]] chill "" reunion of the b...",0.035810,0.991599,1,0,1,24,Successful
3,the story gives ample opportunity for large-sc...,the story gives ample opportunity for large-sc...,0.001556,0.002890,1,1,1,16,Failed
4,"red dragon "" [[never]] cuts corners .","red dragon "" [[mostly]] cuts corners .",0.004349,0.993731,1,0,1,37,Successful
5,fresnadillo has something serious to [[say]] a...,fresnadillo has something serious to [[said]] ...,0.005327,0.162869,1,1,1,29,Failed
6,throws in enough clever and unexpected twists ...,throws in enough clever and unexpected twists ...,0.002805,0.944837,1,0,1,43,Successful
7,weighty and ponderous but every bit as filling...,weighty and ponderous but every bit as filling...,0.939449,0.939449,0,0,1,1,Skipped
8,a real [[audience-pleaser]] that will strike a...,a real [[life]] that will strike a chord with ...,0.000620,0.003631,1,1,1,88,Failed
9,[[generates]] an [[enormous]] feeling of empat...,[[having]] an [[odd]] feeling of empathy for i...,0.000582,0.722759,1,0,1,95,Successful


In [7]:
data['original_text'] = data['original_text'].astype('str').str.replace("[", "", regex=True).astype('str')
data['original_text'] = data['original_text'].astype('str').str.replace("]", "", regex=True).astype('str')
data['perturbed_text'] = data['perturbed_text'].astype('str').str.replace("[", "", regex=True).astype('str')
data['perturbed_text'] = data['perturbed_text'].astype('str').str.replace("]", "", regex=True).astype('str')
data

,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
0,lovingly photographed in the manner of a golde...,lovingly photographed in the manner of a golde...,0.050576,0.672676,1,0,1,46,Successful
1,consistently clever and suspenseful .,consistently annoying and sarcastic .,0.000928,0.996569,1,0,1,35,Successful
2,"it's like a "" big chill "" reunion of the baade...","it's like a "" bad chill "" reunion of the baade...",0.035810,0.991599,1,0,1,24,Successful
3,the story gives ample opportunity for large-sc...,the story gives ample opportunity for large-sc...,0.001556,0.002890,1,1,1,16,Failed
4,"red dragon "" never cuts corners .","red dragon "" mostly cuts corners .",0.004349,0.993731,1,0,1,37,Successful
5,fresnadillo has something serious to say about...,fresnadillo has something serious to said abou...,0.005327,0.162869,1,1,1,29,Failed
6,throws in enough clever and unexpected twists ...,throws in enough clever and unexpected twists ...,0.002805,0.944837,1,0,1,43,Successful
7,weighty and ponderous but every bit as filling...,weighty and ponderous but every bit as filling...,0.939449,0.939449,0,0,1,1,Skipped
8,a real audience-pleaser that will strike a cho...,a real life that will strike a chord with anyo...,0.000620,0.003631,1,1,1,88,Failed
9,generates an enormous feeling of empathy for i...,having an odd feeling of empathy for its chara...,0.000582,0.722759,1,0,1,95,Successful


In [8]:
selected_example = data.loc[data['result_type']=='Successful']
selected_example.reset_index(inplace=True)
selected_example

,index,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
0,0,lovingly photographed in the manner of a golde...,lovingly photographed in the manner of a golde...,0.050576,0.672676,1,0,1,46,Successful
1,1,consistently clever and suspenseful .,consistently annoying and sarcastic .,0.000928,0.996569,1,0,1,35,Successful
2,2,"it's like a "" big chill "" reunion of the baade...","it's like a "" bad chill "" reunion of the baade...",0.035810,0.991599,1,0,1,24,Successful
3,4,"red dragon "" never cuts corners .","red dragon "" mostly cuts corners .",0.004349,0.993731,1,0,1,37,Successful
4,6,throws in enough clever and unexpected twists ...,throws in enough clever and unexpected twists ...,0.002805,0.944837,1,0,1,43,Successful
5,9,generates an enormous feeling of empathy for i...,having an odd feeling of empathy for its chara...,0.000582,0.722759,1,0,1,95,Successful
6,10,exposing the ways we fool ourselves is one hou...,exposing the ways we fool ourselves is one hou...,0.005007,0.994238,1,0,1,58,Successful
7,13,. . . quite good at providing some good old fa...,. . . quite terrible at providing some good ol...,0.002062,0.999460,1,0,1,33,Successful
8,15,scherfig's light-hearted profile of emotional ...,scherfig's low profile of emotional desperatio...,0.001053,0.993670,1,0,1,141,Successful
9,17,the wonderfully lush morvern callar is pure pu...,the wonderfully lush morvern callar is pure pu...,0.046579,0.932213,1,0,1,73,Successful


In [11]:
model_name = "textattack/roberta-base-rotten-tomatoes"

def transferability_percentage(Successful_attack, model_name):
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  def tokenize_function(examples):
      return tokenizer(examples["text"], padding="max_length", truncation=True)
  

  rotten_tomatoes = load_dataset("rotten_tomatoes")
  tokenized_datasets = rotten_tomatoes.map(tokenize_function, batched=True)

  model = AutoModel.from_pretrained(model_name)

  clf = pipeline(task='sentiment-analysis', model=model_name, tokenizer=tokenizer)
  total = 0
  counter = 0
  for a in range(len(Successful_attack)):
    
    ground_truth = Successful_attack['ground_truth_output'][a]
    pred = int(clf.predict(Successful_attack['original_text'][a])[0]['label'][-1])
    if ground_truth == pred:
      total += 1
      perturb = int(clf.predict(Successful_attack['perturbed_text'][a])[0]['label'][-1])
      if perturb != ground_truth:
        counter += 1
  percent = (counter/total)*100
  return percent


In [14]:
print('\n','Transferability percentage:',transferability_percentage(selected_example, model_name))

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Some weights of the model checkpoint at textattack/roberta-base-rotten-tomatoes were not used when initializing RobertaModel: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at textattack/roberta-base-rotten-tomatoes were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint o


 Transferability percentage: 55.55555555555556
